In [56]:
def genLabels(n):
    labels = []
    for i in range(2*n):
        labels.append(i)
    return labels

def genEdges2s(n):
    poset = []
    for i in range(2*n-1):
        if (i%4 == 0):
            poset.append([i, i+1])
        if (i%4 == 1):
            if (i+2 < 2*n):
                poset.append([i, i+2])
        if (i%4 == 2):
            poset.append([i+1, i])
        if (i%4 == 3):
            if (i+2 < 2*n):
                poset.append([i+2, i])
    return poset

def genPoset2s(n):
    return Poset((genLabels(n), genEdges2s(n)))

In [68]:
import itertools
import numpy as np

n = 5

adj = {}
for face in range(2 * n):
    adj[face] = []
    if face - 2 >= 0:
        adj[face].append(face-2)
    if face + 2 < n:
        adj[face].append(face+2)
    if face % 2 == 0:
        adj[face].append(face+1)
    if face % 2 == 1:
        adj[face].append(face-1)

bottom_edges = [(i, i+1) for i in range(0, 2 * n, 2)]
left_edges = [(i, i+2) for i in range(0, 2 * n - 2, 4)] + [(i, i+2) for i in range(1, 2 * n - 2, 4)]
right_edges = [(i, i+2) for i in range(2, 2 * n - 1, 4)] + [(i, i+2) for i in range(3, 2 * n - 2, 4)]

positive_faces = list(range(0, 2*n, 4)) + list(range(3, 2*n, 4))
negative_faces = list(range(2, 2*n, 4)) + list(range(1, 2*n, 4))

# check for crossing violations
def check_crossings(edges, ordering):
    for e1, e2 in itertools.combinations(edges, 2):
        b1 = min(ordering[e1[0]], ordering[e1[1]])
        t1 = max(ordering[e1[0]], ordering[e1[1]])
        b2 = min(ordering[e2[0]], ordering[e2[1]])
        t2 = max(ordering[e2[0]], ordering[e2[1]])

        if b1 < b2 < t1 < t2 or b2 < b1 < t2 < t1:
            # print("CROSSING", e1, e2, b1, t1, b2, t2)
            return False
    return True

# Check that MV is respected: faces with +1 parity (same as the top left) need to be below faces that they share a
# valley fold with, but faces with -1 parity need to satisfy the opposite conditions
def checkMV(faces, parity, assignment, ordering):
    for face in faces:
        for adj_ in adj[face]:
            edge = (min(adj_, face), max(adj_, face))
            # bunch of signs stuff lol
            if np.sign(ordering[face] - ordering[adj_]) != parity * assignment[edge]:
                # print("MV ERROR", face, adj)
                return False
    return True

def verify(assignment, ordering):
    if not checkMV(positive_faces, +1, assignment, ordering) or not checkMV(negative_faces, -1, assignment, ordering):
        return False

    if not check_crossings(bottom_edges, ordering) or not check_crossings(left_edges, ordering) or not check_crossings(right_edges, ordering):
        return False

    return True

def convertToMV(assign):
    edge_assign = {}
    edge_assign[(0,1)] = -1
    for i in range(len(assign)):
        top_left = 2*i
        base = edge_assign[(top_left, top_left+1)]
        if assign[i] == 1:
            edge_assign[(top_left, top_left+2)] = -1*base
            edge_assign[(top_left+1, top_left+3)] = -1*base
            edge_assign[(top_left+2, top_left+3)] = -1*base
        if assign[i] == 2:
            edge_assign[(top_left, top_left+2)] = base
            edge_assign[(top_left+1, top_left+3)] = -1*base
            edge_assign[(top_left+2, top_left+3)] = base
        if assign[i] == 3:
            edge_assign[(top_left, top_left+2)] = base
            edge_assign[(top_left+1, top_left+3)] = base
            edge_assign[(top_left+2, top_left+3)] = -1*base
        if assign[i] == 4:
            edge_assign[(top_left, top_left+2)] = -1*base
            edge_assign[(top_left+1, top_left+3)] = base
            edge_assign[(top_left+2, top_left+3)] = base
    return edge_assign

In [91]:
def countFolds2s(n):
    assignment = convertToMV([2]*n)
    count = 0
    # validWays = []
    P = genPoset2s(n)
    print(P.cover_relations())
    print(assignment)
    for i in P.linear_extension().parent():
        if(verify(assignment, np.argsort(i))):
            # validWays.append(i)
            count+= 1
    return count

In [92]:
print(countFolds2s(4))

[[4, 5], [5, 7], [5, 3], [7, 6], [0, 1], [1, 3], [3, 2]]
{(0, 1): -1, (0, 2): -1, (1, 3): 1, (2, 3): -1, (2, 4): -1, (3, 5): 1, (4, 5): -1, (4, 6): -1, (5, 7): 1, (6, 7): -1, (6, 8): -1, (7, 9): 1, (8, 9): -1}


IndexError: index 8 is out of bounds for axis 0 with size 8